In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
import xgboost as xg 
from matplotlib import pyplot

In [2]:
train_data=pd.read_csv("dataset/train.csv")
test_data=pd.read_csv("dataset/test.csv")
op_raw=pd.read_csv("dataset/test.csv")

In [3]:
def solve(data):
    data['instock_date']=data['instock_date'].str[:4]
    year=pd.get_dummies(data['instock_date'])
    year
    data['year_2014']=year['2014']
    data['year_2015']=year['2015']
    data['year_2016']=year['2016']
    
    Product_Category=pd.get_dummies(data['Product_Category'])
    #data=pd.concat([data,Product_Category],axis=1)
    data['Loyalty_customer']=label_encoder.fit_transform(data['Loyalty_customer'])
    #stall_no=pd.get_dummies(data['Stall_no'])
    #data=pd.concat([data,stall_no],axis=1)
    data=data.drop(['year_2014','Stall_no','Product_Category','Market_Category','Customer_name','Product_id','instock_date'],axis=1)

    data=data.fillna(data.median())
    return data

In [4]:
test=solve(test_data)

In [5]:
train=solve(train_data)


In [6]:
train_y=train['Selling_Price']
train_x=train.drop(['Selling_Price'],axis=1)


In [7]:
import xgboost
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=60,max_depth=8,learning_rate=0.08,min_samples_split=4)
model.fit(train_x,train_y)

c:\users\16063\appdata\local\programs\python\python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\users\16063\appdata\local\programs\python\python37\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[12:20:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=8, min_child_weight=1, min_samples_split=4, missing=None,
             n_estimators=60, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [8]:
res=model.predict(test)
res=res
prod_id=op_raw['Product_id']
op=pd.DataFrame({'Product_id':prod_id,'Selling_Price':res})
op.to_csv('Result.csv')

<h3>Feature Importance<h3>
##Feature Importance

In [9]:
importance = model.feature_importances_
x_data=pd.Series(importance)
y_data=pd.Series(train_x.columns)
imp_data=pd.DataFrame({'ColName':y_data,'Value':x_data})
imp_data.sort_values(by='Value',ascending=False)

,ColName,Value
3,Discount_avail,0.789133
7,Maximum_price,0.146950
6,Minimum_price,0.045291
8,year_2015,0.003486
1,Grade,0.003264
9,year_2016,0.003170
0,Loyalty_customer,0.002788
2,Demand,0.002156
5,charges_2 (%),0.001946
4,charges_1,0.001815


In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
parameter={'learning_rate':[0.07,0.08,0.2,0.3,0.5],'max_depth':[4,5,6,7,8],'min_samples_split':[5,6,7,8]}
clf = XGBRegressor()
Cv_gb=GridSearchCV(estimator=clf,param_grid=parameter,cv=5,verbose=10,n_jobs=-1)

<h3> Selecting Best Parameters using GridSearchCv <h3>

In [14]:
Cv_gb.fit(train_x,train_y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   47.1s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   52.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   58.0s
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  1

[12:24:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.07, 0.08, 0.2, 0.3, 0.5],
                         'max_depth': [4, 5, 6, 7, 8],
        

In [13]:
Cv_gb.best_params_